### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


In [36]:
np.random.rand(ratings_mat.shape[0], 4)

array([[ 0.77742578,  0.48671347,  0.30567401,  0.38286337],
       [ 0.88268661,  0.04896782,  0.23078947,  0.29458565],
       [ 0.61422975,  0.1046405 ,  0.83384186,  0.40972168],
       [ 0.72665376,  0.69938448,  0.32445316,  0.58642573],
       [ 0.60263466,  0.21023076,  0.05545378,  0.33041773],
       [ 0.74153277,  0.43769383,  0.50858375,  0.50146959],
       [ 0.0488912 ,  0.80876769,  0.37063687,  0.49071773],
       [ 0.63015511,  0.40774205,  0.99457006,  0.20899911],
       [ 0.55596292,  0.67439344,  0.65175485,  0.32497426],
       [ 0.98921285,  0.43333832,  0.34139309,  0.81463413],
       [ 0.79734838,  0.04802791,  0.8035761 ,  0.71693872],
       [ 0.03737112,  0.62212899,  0.00574458,  0.6250616 ],
       [ 0.81215383,  0.5900518 ,  0.18915696,  0.54517374],
       [ 0.72424874,  0.80373123,  0.98318489,  0.45976181],
       [ 0.84912598,  0.19714794,  0.00153681,  0.89122537],
       [ 0.4691436 ,  0.0944748 ,  0.36964911,  0.14685073],
       [ 0.95576716,  0.

`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [37]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.sum(np.count_nonzero(ratings_mat, axis = 0)) # total number of ratings in the matrix
    #could also be done as: np.count_nonzero(~np.isnan(ratings_mat)) 
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [40]:
# use your function with 4 latent features, lr of 0.005 and 10 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, learning_rate=.005, iters = 10)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 45.649767
2 		 15.360472
3 		 4.140434
4 		 2.865199
5 		 2.756305
6 		 2.729652
7 		 2.714524
8 		 2.700782
9 		 2.686209
10 		 2.670177


In [41]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.00380239   8.58326719  10.7866752   10.2273735 ]
 [  8.51742837   7.36425538   9.18685445   8.81506799]
 [  7.92866984   6.89770814   8.17271735   7.83903671]
 [  9.48952175   8.38792011   9.97362254   9.6013442 ]
 [  8.79832826   7.56626503   8.86549498   8.3969969 ]
 [  6.61866061   5.77739129   7.3212283    7.01344377]
 [  9.24365303   8.01305745   9.67918835   9.20661184]
 [  8.12737859   7.05925989   8.71052184   8.24108119]
 [  8.39608989   7.49307803   8.83482972   8.56661892]
 [  7.67550161   6.70598475   8.00756854   7.76315942]
 [  8.92896922   7.99738448   9.15252667   8.87056331]
 [  9.46467582   8.48175596   9.989702     9.67012638]
 [  9.23822872   8.29076109   9.60849255   9.29735169]
 [  6.8050254    5.99872154   6.7870112    6.53795132]
 [  9.90141255   8.50509672  10.22792375   9.67468315]
 [  9.73980304   8.50896442  10.23929144   9.74825045]
 [  8.34595902   7.32443145   8.63376871   8.29472881]
 [  7.14389002   6.29019903   7.38247858   7.08592712]
 [  8.2481

**Write your summary here.**

The predicted ratings from the dot product are already starting to look a lot like the original data values even after only 10 iterations. You can see some extreme low values that are not captured well yet. The 5 in the second to last row in the first column is predicted as an 8, and the 4 in the second row and second column is predicted to be a 7. Clearly the model is not done learning, but things are looking good.

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [43]:
#use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=.005, iters = 250)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 41.285544
2 		 13.326755
3 		 3.736758
4 		 2.747318
5 		 2.660895
6 		 2.629836
7 		 2.604677
8 		 2.578483
9 		 2.549570
10 		 2.517352
11 		 2.481476
12 		 2.441653
13 		 2.397648
14 		 2.349300
15 		 2.296541
16 		 2.239413
17 		 2.178083
18 		 2.112836
19 		 2.044078
20 		 1.972305
21 		 1.898078
22 		 1.821984
23 		 1.744599
24 		 1.666453
25 		 1.588014
26 		 1.509684
27 		 1.431810
28 		 1.354706
29 		 1.278694
30 		 1.204123
31 		 1.131401
32 		 1.060990
33 		 0.993402
34 		 0.929162
35 		 0.868771
36 		 0.812654
37 		 0.761121
38 		 0.714332
39 		 0.672289
40 		 0.634834
41 		 0.601674
42 		 0.572410
43 		 0.546576
44 		 0.523671
45 		 0.503196
46 		 0.484671
47 		 0.467659
48 		 0.451769
49 		 0.436665
50 		 0.422062
51 		 0.407724
52 		 0.393464
53 		 0.379133
54 		 0.364623
55 		 0.349857
56 		 0.334792
57 		 0.319413
58 		 0.303728
59 		 0.287772
60 		 0.271601
61 		 0.255288
62 		 0.238924
63 		 0.222610
64 		

In [44]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.00000346  10.00000144  10.00000086   9.99999653]
 [  9.99999369   3.9999973    8.99999836  10.00000637]
 [  7.99999456   8.99999782   9.99999872   5.00000536]
 [  9.00000747   8.00000315  10.00000192   9.99999256]
 [  9.99997639   4.99999037   8.99999431   9.00002354]
 [  6.00001062   4.00000456  10.00000283   5.99998947]
 [  9.00000078   8.00000049  10.00000039   8.99999923]
 [ 10.00001092   5.00000464   9.00000286   7.9999892 ]
 [  6.99998536   7.99999404   9.99999647   8.00001453]
 [  9.00000831   5.00000353   9.00000217   6.99999179]
 [  9.00001483   8.00000611  10.00000365   7.99998525]
 [  9.0000069   10.0000027   10.00000152   8.9999931 ]
 [  9.99999242   8.99999667   9.9999979    8.00000752]
 [  4.9999976    7.99999902   4.99999941   8.00000233]
 [ 10.00000112   8.0000004   10.0000002    9.99999888]
 [  8.99999861   8.99999937   9.99999959  10.00000135]
 [  9.00000142   8.00000054   8.0000003    7.99999858]
 [ 10.00000266   8.00000115   1.00000071   9.99999734]
 [  5.0000

**Write your summary here.**

In this case, we were able to completely reconstruct the item-movie matrix to obtain an essentially 0 mean squared error. I obtained 0 MSE on iteration 174.

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [ ]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

In [45]:
# run SVD on the matrix with the missing value
# use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=.005, iters = 250)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 42.487640
2 		 14.258164
3 		 3.893564
4 		 2.741694
5 		 2.655588
6 		 2.626074
7 		 2.598015
8 		 2.566674
9 		 2.531043
10 		 2.490550
11 		 2.444638
12 		 2.392747
13 		 2.334360
14 		 2.269069
15 		 2.196639
16 		 2.117076
17 		 2.030693
18 		 1.938160
19 		 1.840533
20 		 1.739236
21 		 1.636006
22 		 1.532777
23 		 1.431533
24 		 1.334138
25 		 1.242186
26 		 1.156880
27 		 1.078979
28 		 1.008798
29 		 0.946267
30 		 0.891015
31 		 0.842467
32 		 0.799937
33 		 0.762698
34 		 0.730036
35 		 0.701281
36 		 0.675824
37 		 0.653127
38 		 0.632716
39 		 0.614184
40 		 0.597175
41 		 0.581387
42 		 0.566557
43 		 0.552459
44 		 0.538899
45 		 0.525705
46 		 0.512730
47 		 0.499844
48 		 0.486934
49 		 0.473901
50 		 0.460660
51 		 0.447138
52 		 0.433274
53 		 0.419022
54 		 0.404346
55 		 0.389227
56 		 0.373659
57 		 0.357654
58 		 0.341237
59 		 0.324453
60 		 0.307360
61 		 0.290034
62 		 0.272564
63 		 0.255053
64 		

In [46]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 9.999998473923716:

The actual value for the missing rating is 10.0:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [48]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
# fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=.005, iters = 20)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 0.008145
2 		 0.003755
3 		 0.002570
4 		 0.001981
5 		 0.001611
6 		 0.001352
7 		 0.001159
8 		 0.001010
9 		 0.000891
10 		 0.000794
11 		 0.000714
12 		 0.000647
13 		 0.000591
14 		 0.000543
15 		 0.000502
16 		 0.000467
17 		 0.000436
18 		 0.000410
19 		 0.000387
20 		 0.000367


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [49]:
# Replace each of the comments below with the correct values
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't actually have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [50]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
